# Clean the sample in the SMB_VCV file
Here we remove the Seyfert galaxies and classifications without a known source (No bibcode)

We import the packages first

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.table import Table, join
from astropy.table import unique as tunique

We read the original SMB_VCV file and the table with the Bibcodes for each otype. We rearrange some strings.

In [3]:
# File containing all the otype information
OTTAP = Table.read('BibcodesOtypes.fits', format='fits')
# File with the cross-match between VCV and SMB
TV = Table.read('SMB_VCV.fits', format='fits')

In [4]:
TV['main_id'] = [j.rstrip() for j in TV['main_id']]
TV['Sp'] = [j.rstrip() for j in TV['Sp']]
OTTAP['main_id'] = [j.rstrip() for j in OTTAP['main_id']]
OTTAP['otype_txt'] = [j.rstrip() for j in OTTAP['otype_txt']]
OTTAP['origin'] = [j.rstrip() for j in OTTAP['origin']]

## Remove different redshifts
We remove galaxies where the cross-match shows that the redshift for the galaxies was higher than 0.01 

In [5]:
TV['DiffRed'] = TV['rvz_redshift']-TV['z']
Badz = where(abs(TV['DiffRed']) > 0.01)
TV.remove_rows(Badz)

## Separate origin of the bibcode
We separate the bibcode to simplify querys

In [6]:
def SeparateOrigin(Table):
    """Separate the origin inside the file"""
    Bibcod = np.empty(len(Table), dtype='S19')
    for j, jtex in enumerate(Table):
        if 'bibcode' in jtex['origin']:
            Bibcod[j] = jtex['origin'].split('bibcode=')[-1]
        elif 'from basic data' in jtex['origin']:
            Bibcod[j] = 'CDS'
        elif 'from id' in jtex['origin']:
            Bibcod[j] = 'ID'
        elif jtex['origin'] == '':
            Bibcod[j] = 'UNK'
        else:
            raise NameError('No origin??')
    Table['otype_bibcode'] = Bibcod
    return(Table)

In [7]:
OTTAP = SeparateOrigin(OTTAP)

The total number of galaxies in the sample ({{len(TV)}}) have different otypes, here we center in the `main_id`. Then, we join the files to only have the information of the `main_id` for each galaxy.

In [8]:
AllTypes = join(OTTAP, TV, keys='main_id')
len(unique(AllTypes['main_id']))  # Verification on the number of galaxies

18923

Total number of unknown classifications in SMB is {{sum(AllTypes['otype_bibcode'] == 'UNK')}}

## Dealing with unknown bibcodes and Seyfert types

Some Seyfert sources are not classified and/or the source is unknown. If the source is unknown, but they have a classification in VCV we assume that the classification comes from VCV. 

In [9]:
UKSyGVCV = np.where(np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                           AllTypes['otype_txt_1'] == 'SyG',
                                           AllTypes['otype_txt_2'] == 'SyG',
                                           AllTypes['Sp'] == 'S']))

In [10]:
UKSy1VCV = np.where(np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                           AllTypes['otype_txt_1'] == 'Sy1',
                                           AllTypes['otype_txt_2'] == 'Sy1',
                                           AllTypes['Sp'] == 'S1']))

In [11]:
UKSy2VCV = np.where(np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                           AllTypes['otype_txt_1'] == 'Sy2',
                                           AllTypes['otype_txt_2'] == 'Sy2',
                                           AllTypes['Sp'] == 'S2']))

In [12]:
AllTypes['otype_bibcode'][UKSyGVCV] = 'VCV'
AllTypes['otype_bibcode'][UKSy1VCV] = 'VCV'
AllTypes['otype_bibcode'][UKSy2VCV] = 'VCV'

Then, the total number of unknown classifications in SMB is now {{sum(AllTypes['otype_bibcode'] == 'UNK')}}

We remove the galaxies with a LINER classification in VCV and unknown source in SMB. 

In [13]:
GalaxiesToRemove = AllTypes[np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                                   AllTypes['Sp'] == 'S3',
                                                   AllTypes['otype_txt_1'] == 'Sy2'])]['main_id'].data
LocGTR = [np.where(AllTypes['main_id'] == i)[0] for i in GalaxiesToRemove]
AllTypes.remove_rows(np.concatenate(LocGTR))

If galaxies have an unknown source but are still classified as Seyfert, we send these galaxies from their SMB classification to a Seyfert for further study.

In [14]:
GalaxiesToStudy = concatenate((AllTypes[np.where(np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                                                        AllTypes['otype_txt_1'] == 'Sy2',
                                                                        AllTypes['otype_txt_2'] == 'Sy2']))[0]]['main_id'].data,
                               AllTypes[np.where(np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                                                        AllTypes['otype_txt_1'] == 'Sy1',
                                                                        AllTypes['otype_txt_2'] == 'Sy1']))[0]]['main_id'].data))

In [15]:
GalaxiesToStudy

masked_array(data=['2MASX J18311470-3336085',
                   'EGSIRAC J141515.60+520354.2', 'ESO 323-77',
                   'MCG+03-60-031', 'NAME SMM J141741.90+522823.6',
                   '[VV2006c] J125310.5-091024', '2E  2294', '2E  2628',
                   '2E  3786', '2MASS J00423990+3017514',
                   '2MASS J01341936+0146479', '2MASS J10102753+4132389',
                   '2MASS J12002696+3317286', '2MASX J05064491-1011357',
                   '2MASX J06374318-7538458', '2MASX J07185777+7059209',
                   '2MASX J08420557+0759253', '2MASX J09443702-2633554',
                   '2MASX J10155660-2002268', '2MASX J14555293-3548223',
                   '2MASX J15085291+6814074', '2MASX J16383091-2055246',
                   '2MASX J21033788-0455396', '2MASX J22024516-1304538',
                   '2dFGRS TGN357Z241', '3C 286',
                   '6dFGS gJ043944.9-454043', '6dFGS gJ084628.7-121409',
                   '6dFGS gJ101329.7-283126', '7C 151247.

We re-classify some galaxies from Sy1 or Sy2 to SyG , to study later on. Then, we remove galaxies that are not useful. And finally, we create a flag for the type of emission

In [16]:
Reclass = [np.where(TV['main_id'] == gal)[0][0] for gal in GalaxiesToStudy]
TV['otype_txt'][Reclass] = 'SyG'

LocGTR_TV = [np.where(TV['main_id'] == i)[0] for i in GalaxiesToRemove]

TV.remove_rows(np.concatenate(LocGTR_TV))

From the rest of this work we are going to assume that S1n are also S1 galaxies. Only three galaxies classified as S1n in VCV are classified as Sy2 in SIMBAD. We are going to include this three galaxies in the unclassified sample.

In [17]:
Reclass2 = np.where(np.logical_and(TV['Sp'] == 'S1n', TV['otype_txt'] == 'Sy2'))
print(TV[Reclass2]['main_id'])
TV['Sp'][Reclass2] = 'S'
Reclass3 = np.where(TV['Sp'] == 'S1n')
TV['Sp'][Reclass3] = 'S1'

        main_id        
-----------------------
2MASX J10194946+3322041
2MASS J09455439+4238399
2MASX J23383708-0028105


We additionally save the information of the galaxies that where reclassified as S1 from S1n for further analysis.

In [21]:
TV[Reclass3]['main_id','otype_txt'].write('NLS1_reclass.txt', format='ascii')

## Create a new VCV SMB file
Here we organize and save the data to be used later. We get in total {{len(TV)}} galaxies in the sample

In [18]:
# We named TAP as these come from TAP Simbad
TV.write('VCV_TAP_otype.txt', format='ascii')

## Final numbers from the otypes
##### Part TABLE 3
We compute the final numbers about where the classifications come from. We take into account the reclassification that we use before for specific galaxies.

In [19]:
ReclassS1 = np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                   AllTypes['otype_txt_1'] == 'Sy1',
                                   AllTypes['otype_txt_2'] == 'Sy1'])

ReclassS2 = np.logical_and.reduce([AllTypes['otype_bibcode'] == 'UNK',
                                   AllTypes['otype_txt_1'] == 'Sy2',
                                   AllTypes['otype_txt_2'] == 'Sy2'])

Data from Seyfert 1 galaxies comes predominantly (95%) from 5 works 

In [20]:
print('The total number of Seyfert 1 is:', 
      len(AllTypes[np.logical_and(AllTypes['otype_txt_1'] == 'Sy1', AllTypes['otype_txt_2'] == 'Sy1')])-sum(ReclassS1))

The total number of Seyfert 1 is: 13760


Here we show the bibcode for the first 5 contributions to the Seyfert 1 sample

In [21]:
S1Otyp = np.logical_xor(np.logical_and(AllTypes['otype_txt_1'] == 'Sy1', AllTypes['otype_txt_2'] == 'Sy1'), ReclassS1)
AllTypes[S1Otyp].to_pandas().groupby('otype_bibcode').size().sort_values(ascending=False).head(5)

otype_bibcode
b'CDS'                    5566
b'2014ApJ...788...45T'    2784
b'2015ApJS..219....1O'    2001
b'2006ApJS..166..128Z'    1783
b'2017ApJS..229...39R'     988
dtype: int64

In [22]:
## Fractional contribution from those 5 works
(sum(AllTypes[S1Otyp].to_pandas().groupby('otype_bibcode').size().sort_values(ascending=False).head(5))) / \
    (len(AllTypes[np.logical_and(AllTypes['otype_txt_1'] == 'Sy1', AllTypes['otype_txt_2'] == 'Sy1')])-sum(ReclassS1))

0.9536337209302326

Data from Seyfert 2 galaxies comes predominantly (97%) from 3 works

In [23]:
print('The total number of Seyfert 2 is:', 
      len(AllTypes[np.logical_and(AllTypes['otype_txt_1'] == 'Sy2', AllTypes['otype_txt_2'] == 'Sy2')])-sum(ReclassS2))

The total number of Seyfert 2 is: 5040


Here we show the bibcode for the first 3 contributions to the Seyfert 2 sample

In [24]:
S2Otyp = np.logical_xor(np.logical_and(AllTypes['otype_txt_1'] == 'Sy2', AllTypes['otype_txt_2'] == 'Sy2'), ReclassS2)
AllTypes[S2Otyp].to_pandas().groupby('otype_bibcode').size().sort_values(ascending=False).head(3)

otype_bibcode
b'CDS'                    2845
b'2014ApJ...788...45T'    1239
b'VCV'                     787
dtype: int64

In [25]:
## Fractional contribution from those 3 works
(sum(AllTypes[S2Otyp].to_pandas().groupby('otype_bibcode').size().sort_values(ascending=False).head(3))) / \
    (len(AllTypes[np.logical_and(AllTypes['otype_txt_1'] == 'Sy2', AllTypes['otype_txt_2'] == 'Sy2')])-sum(ReclassS2))

0.966468253968254

Finally, data from the unclassified Seyfert galaxies comes predominantly (92%) from 3 works

In [26]:
print('The total number of Seyfert 2 is:', 
      len(AllTypes[np.logical_and(AllTypes['otype_txt_1'] == 'SyG', AllTypes['otype_txt_2'] == 'SyG')])+sum(np.logical_or(ReclassS1, ReclassS2)))

The total number of Seyfert 2 is: 121


We show the bibcode for the first 3 contributions to the unclassified Seyfert sample

In [27]:
SGOtyp = np.logical_xor(np.logical_and(AllTypes['otype_txt_1'] == 'SyG', AllTypes['otype_txt_2'] == 'SyG'), np.logical_or(ReclassS1, ReclassS2))
AllTypes[SGOtyp].to_pandas().groupby('otype_bibcode').size().sort_values(ascending=False).head(3)

otype_bibcode
b'CDS'    57
b'UNK'    49
b'VCV'     5
dtype: int64

In [28]:
## Fractional contribution from those 3 works
(sum(AllTypes[SGOtyp].to_pandas().groupby('otype_bibcode').size().sort_values(ascending=False).head(3))) / \
    (len(AllTypes[np.logical_and(AllTypes['otype_txt_1'] == 'SyG',
                                 AllTypes['otype_txt_2'] == 'SyG')])+sum(np.logical_or(ReclassS1, ReclassS2)))

0.9173553719008265

We notice that almost 45% of the classifications come from the CDS basic data

In [29]:
(sum(AllTypes[S1Otyp]['otype_bibcode'] == 'CDS')+sum(AllTypes[S2Otyp]['otype_bibcode'] == 'CDS')+sum(AllTypes[SGOtyp]['otype_bibcode'] == 'CDS'))/len(AllTypes[np.logical_or.reduce([np.logical_and(AllTypes['otype_txt_1'] == 'Sy2', AllTypes['otype_txt_2'] == 'Sy2'),
                                                                                                                                                                                     np.logical_and(
                                                                                                                                                                                         AllTypes['otype_txt_1'] == 'Sy1', AllTypes['otype_txt_2'] == 'Sy1'),
                                                                                                                                                                                     np.logical_and(AllTypes['otype_txt_1'] == 'SyG', AllTypes['otype_txt_2'] == 'SyG')])])

0.44754505575815234

##### Notebook info

In [30]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy,pandas

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.2
pandas : 1.2.0

re        : 2.2.1
matplotlib: 3.2.2
json      : 2.0.9
autopep8  : 1.5.4
numpy     : 1.19.5
sys       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
logging   : 0.5.1.2

Watermark: 2.1.0

